In [6]:
#import
import pandas as pd
import numpy as np
import hddm
from scipy import stats
import seaborn as sns
import matplotlib.pyplot as plt
import pymc
import kabuki
sns.set(style="white")
%matplotlib inline
from tqdm import tqdm
import warnings
warnings.filterwarnings("ignore", category=np.VisibleDeprecationWarning)

In [7]:
# hddm.generate.gen_rand_rlddm_data(a=1,t=0.3,alpha=0.8,scaler=1,subjs=1,split_by=0,size=20)

In [8]:
data = hddm.generate.gen_rl_data_MAB_with_RT(num_trials=20, mu=[1,0], mu_sd=0.1, a=1, t=0.3, scaler=1)

#data['rt'] = 0.5
data['subj_idx'] = 111
data['q_init'] = 0.5
data['split_by'] = 0

data.head()

>  [0.5, 0.5] 0.0 0 0.5559999999980818
>  [1.0621495250176634, 0.5] 0.5621495250176634 0 0.451999999998047
>  [0.9859596108909983, 0.5] 0.48595961089099826 1 0.6110000000021076
>  [0.9859596108909983, -0.02014827366977401] 1.0061078845607723 1 0.5920000000021013
>  [0.9859596108909983, 0.04474662649395056] 0.9412129843970477 1 0.5650000000020923
>  [0.9859596108909983, -0.023397550930715993] 1.0093571618217143 1 0.5170000000020762
>  [0.9859596108909983, -0.05601333847310886] 1.0419729493641072 1 0.5860000000020993
>  [0.9859596108909983, -0.046193034814320896] 1.0321526457053192 1 0.3530000000020214
>  [0.9859596108909983, -0.055474283139117626] 1.041433894030116 1 0.37100000000202743
>  [0.9859596108909983, -0.04459712188247706] 1.0305567327734753 1 0.46900000000206016
>  [0.9859596108909983, -0.027754972788312026] 1.0137145836793102 1 0.3820000000020311
>  [0.9859596108909983, -0.021818283672269803] 1.007777894563268 0 0.43799999999804234
>  [0.9637219222738643, -0.02181828367226980

,trial,response,feedback,rt,subj_idx,q_init,split_by
0,0,0,1.062150,0.556,111,0.5,0
1,1,0,0.909770,0.452,111,0.5,0
2,2,1,-0.020148,0.611,111,0.5,0
3,3,1,0.109642,0.592,111,0.5,0
4,4,1,-0.159686,0.565,111,0.5,0


In [ ]:
#run the model by calling hddm.Hrl (instead of hddm.HDDM for normal model and hddm.HDDMrl for rlddm-model)
m_rl = hddm.HDDMrl(data, params=['scaler'])

#set sample and burn-in
m_rl.sample(500,burn=200,dbname='traces.db',db='pickle')
#print stats to get an overview of posterior distribution of estimated parameters
m_rl.print_stats()

Parameter estimates from the pure RL-model are a bit different compared to the RLDDM. This is to be expected as probability of choice in DDM is dependent both on the decsision threshold and the scaled difference in q-values, whereas the RL model only uses the scaled difference in q-values. 

In [ ]:
m_rl.plot_posteriors()

In [ ]:
alpha, beta = m_rl.nodes_db.node[['alpha', 'beta']]
samples = {'alpha':alpha.trace(),'beta':beta.trace()}
samp = pd.DataFrame(data=samples)

def corrfunc(x, y, **kws):
    r, _ = stats.pearsonr(x, y)
    ax = plt.gca()
    ax.annotate("r = {:.2f}".format(r),
                xy=(.1, .9), xycoords=ax.transAxes)

g = sns.PairGrid(samp, palette=["red"])
g.map_upper(plt.scatter, s=10)
g.map_diag(sns.distplot, kde=False)
g.map_lower(sns.kdeplot, cmap="Blues_d")
g.map_lower(corrfunc)